In [288]:
import json
import numpy as np
import networkx as nx
import gurobipy as gp
from gurobipy import *
import random
import copy
np.random.seed(0)
from tqdm import tqdm
import time
import collections
import scipy

In [249]:
print(gp.gurobi.version())

(10, 0, 1)


## Cascade Functions

In [250]:
def simulate_cascade(G, p_lb=0, p_ub=1):
    """
    Simulate a cascade in graph by flipping coins for all edges in G to construct a subgraph G'
    
    Parameters:
    G (networkx.Graph): The graph on which to run the model.
    
    Returns:
    G_prime (networkx.Graph): A subgraph of G
    """
    
    G_prime = G.copy()
    
    removed_edges = [e for e in G.edges if np.random.uniform() < G[e[0]][e[1]]['weight']]
    
    G_prime.remove_edges_from(removed_edges)
    
    return G_prime

In [251]:
def simulate_fixed_y_cascade(G, ys):
    '''
    Simulate a cascade with fixed initial set of nodes
    '''
    # Set all nodes as not activated
    activated = set()
    # Add the seed nodes to the activated set
    activated.update(set(ys))
    # Initialize the newly activated nodes list with the seed nodes
    newly_activated = set(ys)
    
    # Run the model until there are no more newly activated nodes
    while len(newly_activated) != 0:
        temp_activated = set()
        for v in newly_activated:
            # Check for each successor if it gets activated
            for w in G.successors(v):
                if w not in activated:
                    u = np.random.uniform()
                    if u < G[v][w]['weight']:
                        temp_activated.add(w)
        # Add newly activated nodes to the activated set
        newly_activated = temp_activated
        activated.update(newly_activated)
    
    return len(activated)

In [252]:
def simulate_fixed_y_fixed_cascade(sub_G, ys):
    '''
    Simulate a cascade with fixed initial set of nodes
    '''
    # Set all nodes as not activated
    activated = set()
    # Add the seed nodes to the activated set
    activated.update(set(ys))
    # Initialize the newly activated nodes list with the seed nodes
    newly_activated = set(ys)
    
    # Run the model until there are no more newly activated nodes
    while len(newly_activated) != 0:
        temp_activated = set()
        for v in newly_activated:
            # Check for each successor if it gets activated
            for w in sub_G.successors(v):
                temp_activated.add(w)
        # Add newly activated nodes to the activated set
        newly_activated = temp_activated
        activated.update(newly_activated)
    
    return len(activated)

## MIP Model

In [253]:
# def solve_max_spread(graphs, B, C):
#     """
#     Solves the MIP model for maximizing the spread of cascades using network design as presented in the paper
#     "Maximizing the Spread of Cascades Using Network Design" by Sheldon et al. for the given graph and budget.
    
#     Parameters:
#     graphs (list [networkx.Graph]): List of training cascades graphs, which are subgraphs of the original network
#     budget (int): The budget of edges that can be added to the graph.
#     costs (list): Cost of each action
#     """
#     # Initialize the MIP model
#     model = gp.Model('maximize_spread_cascades')
    
#     # Number of training cascades
#     N = len(graphs)

#     # Initialize the decision variables
#     x = {}
#     y = {}
    
#     nodes = np.sort(list(graphs[0].nodes))
    
#     for k in range(N):
#         for v in nodes:
#             x[k, v] = model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = 1, name=f'x_{k}_{v}')
#     for v in nodes:
#         y[v] = model.addVar(vtype=GRB.BINARY, name=f'y_{v}') # Asssuming each action corresponds to purchasing one node.

#     # Initialize the objective function to maximize the expected spread
#     obj = 1/N * gp.quicksum(x[k, v] for k in range(N) for v in nodes)
#     model.setObjective(obj, GRB.MAXIMIZE)
    
#     # Add the budget constraint
#     model.addConstr(gp.quicksum(C[v]*y[v] for v in nodes) <= B, name='budget_constraint')
    
#     # Add the edge constraints
#     for k in range(N):
#         for v in nodes:
#             preds = set(graphs[k].predecessors(v))
#             if len(preds) != 0:
#                 model.addConstr((x[k, v] <= gp.quicksum(x[k, u] for u in preds)), name=f'edge_constraint_{k}_{v}')

#     # Add the coverage constraint
#     model.addConstrs((x[k, v] <= y[v] for v in nodes for k in range(N)), name=f'coverage_constraint_{k}_{v}')
    
#     model.update()

#     # Optimize the model
#     model.optimize()
    
#     # Get Objective value
#     obj = model.getObjective().getValue()
#     ys = [v for v in nodes if y[v].x == 1]
#     return model, obj, ys

In [254]:
def solve_max_spread(pres, B, C, N, nodes):
    """
    Parameters:
    pres (dict): Set of all predecessor nodes (including the node itself) pres[k, i] who can activate node i in scenario k.
    B (int): The budget of edges that can be added to the graph.
    C (list): Cost of each action
    N (int): Number of training cascades
    """
    # Initialize the MIP model
    model = gp.Model('maximize_spread_cascades')

    # Initialize the decision variables
    x = {}
    y = {}
        
    for k in range(N):
        for v in nodes:
            x[k, v] = model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = 1, name=f'x_{k}_{v}')
    for v in nodes:
        y[v] = model.addVar(vtype=GRB.BINARY, name=f'y_{v}') # Asssuming each action corresponds to purchasing one node.

    # Initialize the objective function to maximize the expected spread
    obj = 1/N * gp.quicksum(x[k, v] for k in range(N) for v in nodes)
    model.setObjective(obj, GRB.MAXIMIZE)
    
    # Add the budget constraint
    model.addConstr(gp.quicksum(C[v]*y[v] for v in nodes) <= B, name='budget_constraint')

    # Add the coverage constraint
    model.addConstrs((x[k, v] <= gp.quicksum(y[j] for j in pres[k, v]) for v in nodes for k in range(N)), 
                     name=f'coverage_constraint_{k}_{v}')
    
    model.update()

    # Optimize the model
    model.optimize()
    
    # Get Objective value
    obj = model.getObjective().getValue()
    ys = [v for v in nodes if y[v].x == 1]
    return model, obj, ys

In [255]:
def find_pres(graphs):
    '''
    Return set of all predecessor nodes who can activate node i in scenario k
    '''
    pres = {}
    for k in range(len(graphs)):
        G = graphs[k]
        print(f'finding predecessors for scenario {k+1}')
        for i in G.nodes:
            pres[k, i] = set([i])
            # Start BFS from node i
            visited = set()
            queue = collections.deque([i])
            while queue:
                node = queue.popleft()
                if node not in visited:
                    visited.add(node)
                    for predecessor in G.predecessors(node):
                        if predecessor not in visited:
                            pres[k,i].add(predecessor)
                            queue.append(predecessor)
    return pres

## SAA Algo

In [256]:
def SAA_phase1(input_graph, B, C, num_sample = 50, num_train = 10):
    sol_lst = []
    obj_ub_lst = []
    for i in range(num_sample):
        print(f'currently in SAA phase 1 {i+1}th sample')
        np.random.seed(i)
        # For each sample, generate num_train of training cascades.
        graphs = [simulate_cascade(input_graph) for _ in range(num_train)]
        # Find predecessors of each node in each realization.
        pres = find_pres(graphs)
        model, obj, ys = solve_max_spread(pres, B, C, num_train, input_graph.nodes)
        sol_lst.append(ys)
        obj_ub_lst.append(obj)
        
    return sol_lst, obj_ub_lst

In [257]:
def SAA_phase2(input_graph, sol_lst, num_valid = 500, num_test = 500):
    print('SAA phase 2')
    # Choose the best solution by re-estimating using num_valid training cascades.
    obj_avg_lst = [np.mean([simulate_fixed_y_cascade(input_graph, sol)
                            for _ in range(num_valid)]) for sol in tqdm(sol_lst)]

    y_opt = sol_lst[np.argmax(obj_avg_lst)]
    # Generate num_test replications for testing the best solution.
    obj_lb_lst = [simulate_fixed_y_cascade(input_graph, y_opt) for _ in range(num_test)]
    
    return y_opt, obj_lb_lst

In [299]:
def main_algo(G, B, C, num_sample, num_train, num_valid, num_test):
    start_time = time.perf_counter()
    
    sol_lst, obj_ub_lst = SAA_phase1(G, B, C, num_sample = num_sample, num_train = num_train)
    y_opt, obj_lb_lst = SAA_phase2(G, sol_lst, num_valid = num_valid, num_test = num_test)
    
    end_time = time.perf_counter()
    cpu_time = end_time - start_time
    print("CPU time:", cpu_time)

    t_critical = scipy.stats.t.ppf(0.975, num_sample-1)
    ub_mean = np.mean(obj_ub_lst)
    ub_error = t_critical*(np.std(obj_ub_lst)/np.sqrt(num_sample))
    ub_CI_lower = ub_mean - ub_error 
    ub_CI_upper = ub_mean + ub_error 
    print("95% Confidence Interval of upper bound estimate: [{:.2f}, {:.2f}]".format(ub_CI_lower, ub_CI_upper))

    lb_mean = np.mean(obj_lb_lst)
    lb_error = 1.96*(np.std(obj_lb_lst)/np.sqrt(num_test))
    lb_CI_lower = lb_mean-lb_error
    lb_CI_upper = lb_mean+lb_error
    print("95% Confidence Interval of lower bound estimate: [{:.2f}, {:.2f}]".format(lb_CI_lower, lb_CI_upper))
    
    opt_gap = round(100 * (ub_mean - lb_mean) / ub_mean, 3)
    print("Estimated optimality gap is: ", opt_gap, "%")
    

    return len(y_opt), round(ub_mean, 3), round(ub_error,3), round(lb_mean,3), round(lb_error,3), round(cpu_time,3), opt_gap

## Experiment

### 1. Random DAG

In [259]:
# Create a random Gnp graph.
G = nx.gnp_random_graph(5, 0.5, seed=0, directed=True)

# Create a directed acyclic graph by only keeping edges that point from lower (newer) indices to higher (older).
DAG = nx.DiGraph()
DAG.add_nodes_from(G.nodes)
DAG.add_edges_from([(u,v) for (u,v) in G.edges() if u<v])
print('Is the graph acyclic:', nx.is_directed_acyclic_graph(DAG))
print('Num edges: ', len(DAG.edges))

Is the graph acyclic: True
Num edges:  5


In [260]:
# Add random activation probabilities for each edge
np.random.seed(0)
for e in DAG.edges():
    DAG[e[0]][e[1]]['weight'] = np.random.uniform(0, 1)

In [261]:
# Input parameters - budget and action cost
B = 300
np.random.seed(0)
# Cost of each node is uniformly distribution from (0, 100)
# C = [np.random.uniform(0, 100) for _ in range(len(DAG))]
C = 100 * np.ones(len(DAG))

Test 1.1: 

In [262]:
np.random.seed(0)
graphs = [simulate_cascade(DAG) for _ in range(1)]
pres = find_pres(graphs)

finding predecessors for scenario 1


In [263]:
graphs[0].edges

OutEdgeView([(0, 3), (0, 4), (1, 2), (1, 4), (3, 4)])

In [264]:
pres

{(0, 0): {0},
 (0, 1): {1},
 (0, 2): {1, 2},
 (0, 3): {0, 3},
 (0, 4): {0, 1, 3, 4}}

In [265]:
model, obj, ys = solve_max_spread(pres, B, C, 1, graphs[0].nodes)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 10 columns and 20 nonzeros
Model fingerprint: 0xb43dba0d
Variable types: 5 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+02, 3e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 6 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 5 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%


In [266]:
simulate_fixed_y_fixed_cascade(graphs[0], ys)

5

### 2. Reddit Interaction Network

Refer to http://snap.stanford.edu/data/web-RedditNetworks.html

In [267]:
# read in networks for subreddit /r/politics
with open("politics.json") as fp:
    month_nets = json.load(fp)

# see who the user sn00gan replied to in the first month
print(month_nets[0]["sn00gan"])

['TedTheGreek_Atheos', 'ptwonline', 'sn00gan', 'TiiziiO', 'caged_raptor']


In [268]:
# Create graph
G2 = nx.DiGraph()
for i in range(len(month_nets)):
    for k,v in month_nets[i].items():
        for vs in v:
            G2.add_edge(k, vs)

In [269]:
len(G2.edges)

1688750

In [270]:
len(G2.nodes)

119780

**Potential variables: number of edges (size of network), budget, num_train**

**Performance measures: seed set size, CPU time, optimality gap, lb CI, ub CI**

Experiment 2.1: Varying bugdet from 20 to 200. Num edges - fixed as 2000; num trains - fixed as 20.

In [300]:
# Choose a random subgraph with the specified number of edges
num_edges = 2000
random.seed(10)
subgraph_edges = random.sample(G2.edges(), num_edges)
sub_G2 = G2.edge_subgraph(subgraph_edges)
print('Num nodes:', len(sub_G2))
print('Num edges:', len(sub_G2.edges))

C:\Users\USER\AppData\Local\Temp\ipykernel_18940\1613497201.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subgraph_edges = random.sample(G2.edges(), num_edges)


Num nodes: 3225
Num edges: 2000


In [301]:
# Add random activation probabilities for each edge
np.random.seed(0)
for e in sub_G2.edges():
    sub_G2[e[0]][e[1]]['weight'] = np.random.uniform(0, 1)

np.random.seed(0)
# Cost of each node is uniformly distribution from (0, 100)
C = {v: np.random.uniform(0, 100) for v in sub_G2.nodes}

In [302]:
# Run the algo with B in the set {20, 40, 60, 80, 100, 120, 140, 160, 180, 200}
num_sample = 50
num_train = 10
num_valid = 500
num_test = 500

outputs = []
with open('output.txt', 'w') as f:
    for B in [20, 40, 60, 80, 100, 120, 140, 160, 180, 200]:
        print(f'running experiment B = {B}....')
        output = main_algo(sub_G2, B, C, num_sample, num_train, num_valid, num_test)
        f.write(f'Output for B = {B}:\n')
        for item in output:
            f.write(str(item))
            f.write('\n')
        f.write('\n')
        outputs.append(output)

running experiment B = 20....
currently in SAA phase 1 1th sample
finding predecessors for scenario 1
finding predecessors for scenario 2
finding predecessors for scenario 3
finding predecessors for scenario 4
finding predecessors for scenario 5
finding predecessors for scenario 6
finding predecessors for scenario 7
finding predecessors for scenario 8
finding predecessors for scenario 9
finding predecessors for scenario 10
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 32251 rows, 35475 columns and 80363 nonzeros
Model fingerprint: 0xa3ad7e7b
Variable types: 32250 continuous, 3225 integer (3225 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+02]
  Objective range  [1e-01, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objecti

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 32251 rows, 35475 columns and 78911 nonzeros
Model fingerprint: 0xb007dbc3
Variable types: 32250 continuous, 3225 integer (3225 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+02]
  Objective range  [1e-01, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 32147 rows and 34991 columns
Presolve time: 0.04s
Presolved: 104 rows, 484 columns, 722 nonzeros
Variable types: 0 continuous, 484 integer (484 binary)
Found heuristic solution: objective 42.5000000

Root relaxation: objective 5.879572e+01, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | 

Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 32251 rows, 35475 columns and 79007 nonzeros
Model fingerprint: 0x24644efe
Variable types: 32250 continuous, 3225 integer (3225 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+02]
  Objective range  [1e-01, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 32138 rows and 34974 columns
Presolve time: 0.04s
Presolved: 113 rows, 501 columns, 759 nonzeros
Variable types: 0 continuous, 501 integer (501 binary)
Found heuristic solution: objective 26.7000000

Root relaxation: objective 5.714153e+01, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   57.14153    0    2   26.70000   57.14153   114%     -    0s
H    0     0              

Coefficient statistics:
  Matrix range     [7e-03, 1e+02]
  Objective range  [1e-01, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 32130 rows and 34961 columns
Presolve time: 0.04s
Presolved: 121 rows, 514 columns, 789 nonzeros
Variable types: 0 continuous, 514 integer (514 binary)
Found heuristic solution: objective 40.5000000

Root relaxation: objective 5.655773e+01, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   56.55773    0    1   40.50000   56.55773  39.6%     -    0s
H    0     0                      56.2000000   56.55773  0.64%     -    0s

Cutting planes:
  Gomory: 1
  Cover: 1
  MIR: 1
  StrongCG: 1

Explored 1 nodes (1 simplex iterations) in 0.08 seconds (0.05 work units)
Thread count was 12 (of 12 available processors)



Presolve time: 0.04s
Presolved: 116 rows, 503 columns, 767 nonzeros
Variable types: 0 continuous, 503 integer (503 binary)
Found heuristic solution: objective 34.2000000

Root relaxation: objective 5.803715e+01, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   58.03715    0    1   34.20000   58.03715  69.7%     -    0s
H    0     0                      57.3000000   58.03715  1.29%     -    0s
H    0     0                      57.7000000   58.03715  0.58%     -    0s

Cutting planes:
  Cover: 1

Explored 1 nodes (2 simplex iterations) in 0.08 seconds (0.05 work units)
Thread count was 12 (of 12 available processors)

Solution count 4: 57.7 57.3 34.2 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.770000000000e+01, best bound 5.770000000000e+01, gap 0.0000%
currently in SAA phase 1 18th sample
finding predeces

...

     0     0  161.92612    0    1  161.80000  161.92612  0.08%     -    0s
     0     0  161.92102    0    2  161.80000  161.92102  0.07%     -    0s
     0     0  161.90653    0    2  161.80000  161.90653  0.07%     -    0s
     0     0     cutoff    0       161.80000  161.80000  0.00%     -    0s

Cutting planes:
  Cover: 1
  StrongCG: 1
  RLT: 1

Explored 1 nodes (23 simplex iterations) in 0.17 seconds (0.09 work units)
Thread count was 12 (of 12 available processors)

Solution count 5: 161.8 161.7 161.4 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 1.618000000000e+02, best bound 1.618000000000e+02, gap 0.0000%
currently in SAA phase 1 43th sample
finding predecessors for scenario 1
finding predecessors for scenario 2
finding predecessors for scenario 3
finding predecessors for scenario 4
finding predecessors for scenario 5
finding predecessors for scenario 6
finding predecessors for scenario 7
finding predecessors for scenario 8
finding predecessors for scenari


     0     0  162.75629    0   11   11.20000  162.75629  1353%     -    0s
H    0     0                     162.5000000  162.75629  0.16%     -    0s
H    0     0                     162.6000000  162.75629  0.10%     -    0s
     0     0  162.74016    0   13  162.60000  162.74016  0.09%     -    0s

Explored 1 nodes (14 simplex iterations) in 0.16 seconds (0.09 work units)
Thread count was 12 (of 12 available processors)

Solution count 4: 162.6 162.5 11.2 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.626000000000e+02, best bound 1.626000000000e+02, gap 0.0000%
currently in SAA phase 1 47th sample
finding predecessors for scenario 1
finding predecessors for scenario 2
finding predecessors for scenario 3
finding predecessors for scenario 4
finding predecessors for scenario 5
finding predecessors for scenario 6
finding predecessors for scenario 7
finding predecessors for scenario 8
finding predecessors for scenario 9
finding predecessors for scenario 10
Gurobi Optimi


Explored 1 nodes (18 simplex iterations) in 0.13 seconds (0.08 work units)
Thread count was 12 (of 12 available processors)

Solution count 4: 162.9 162.8 12.5 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.629000000000e+02, best bound 1.629000000000e+02, gap 0.0000%
SAA phase 2


100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [04:35<00:00,  5.51s/it]


CPU time: 530.8334811999957
95% Confidence Interval of upper bound estimate: [163.01, 163.84]
95% Confidence Interval of lower bound estimate: [146.80, 147.67]
Estimated optimality gap is:  9.904 %


Experiment 2.2: Varying num_train from 5 to 100. Num edges - fixed as 2000; B = 200.

In [303]:
# Run the algo with num_train in [5, 10, 15, 20, 30, 50, 75, 100]
num_sample = 20
num_valid = 500
num_test = 500
B = 200

outputs = []
with open('output2.txt', 'w') as f:
    for num_train in [5, 10, 15, 20, 30, 50, 75, 100]:
        print(f'running experiment num_train = {num_train}....')
        output = main_algo(sub_G2, B, C, num_sample, num_train, num_valid, num_test)
        f.write(f'Output for num_train = {num_train}:\n')
        for item in output:
            f.write(str(item))
            f.write('\n')
        f.write('\n')
        outputs.append(output)

running experiment num_train = 5....
currently in SAA phase 1 1th sample
finding predecessors for scenario 1
finding predecessors for scenario 2
finding predecessors for scenario 3
finding predecessors for scenario 4
finding predecessors for scenario 5
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16126 rows, 19350 columns and 42578 nonzeros
Model fingerprint: 0xead805a5
Variable types: 16125 continuous, 3225 integer (3225 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+02]
  Objective range  [2e-01, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 13935 rows and 13943 columns
Presolve time: 0.05s
Presolved: 2191 rows, 5407 columns, 11154 nonzeros
Variable types: 0 continuous, 5407 integer

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16126 rows, 19350 columns and 41143 nonzeros
Model fingerprint: 0xcb978074
Variable types: 16125 continuous, 3225 integer (3225 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+02]
  Objective range  [2e-01, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 14320 rows and 14329 columns
Presolve time: 0.04s
Presolved: 1806 rows, 5021 columns, 9435 nonzeros
Variable types: 0 continuous, 5021 integer (5021 binary)
Found heuristic solution: objective 11.4000000

Root relaxation: objective 1.631505e+02, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  166.52449    0    2   13.80000  166.52449  1107%     -    0s
H    0     0                     166.2000000  166.52449  0.20%     -    0s
H    0     0                     166.4000000  166.52449  0.07%     -    0s

Cutting planes:
  Cover: 1

Explored 1 nodes (15 simplex iterations) in 0.10 seconds (0.06 work units)
Thread count was 12 (of 12 available processors)

Solution count 4: 166.4 166.2 13.8 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.664000000000e+02, best bound 1.664000000000e+02, gap 0.0000%
currently in SAA phase 1 10th sample
finding predecessors for scenario 1
finding predecessors for scenario 2
finding predecessors for scenario 3
finding predecessors for scenario 4
finding predecessors for scenario 5
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16126 rows, 19350 columns and 41124 nonzeros
Model fingerprint: 0x1b2e961a
Variable types: 16125 continuous, 3225 integer (3225 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+02]
  Objective range  [2e-01, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 14362 rows and 14373 columns
Presolve time: 0.04s
Presolved: 1764 rows, 4977 columns, 9218 nonzeros
Variable types: 0 continuous, 4977 integer (4977 binary)
Found heuristic solution: objective 11.2000000

Root relaxation: objective 1.647546e+02, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int


Root relaxation: objective 1.628811e+02, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  162.88115    0    3   12.60000  162.88115  1193%     -    0s
H    0     0                     162.0000000  162.88115  0.54%     -    0s
H    0     0                     162.6000000  162.88115  0.17%     -    0s
     0     0  162.85028    0    2  162.60000  162.85028  0.15%     -    0s
     0     0  162.85028    0    1  162.60000  162.85028  0.15%     -    0s
     0     0  162.83213    0    1  162.60000  162.83213  0.14%     -    0s

Cutting planes:
  RLT: 2

Explored 1 nodes (12 simplex iterations) in 0.14 seconds (0.07 work units)
Thread count was 12 (of 12 available processors)

Solution count 4: 162.6 162 12.6 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.626000000000e+02, best bound 1.626000000000e+02, gap 0.0000%

...

finding predecessors for scenario 87
finding predecessors for scenario 88
finding predecessors for scenario 89
finding predecessors for scenario 90
finding predecessors for scenario 91
finding predecessors for scenario 92
finding predecessors for scenario 93
finding predecessors for scenario 94
finding predecessors for scenario 95
finding predecessors for scenario 96
finding predecessors for scenario 97
finding predecessors for scenario 98
finding predecessors for scenario 99
finding predecessors for scenario 100
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 322501 rows, 325725 columns and 760560 nonzeros
Model fingerprint: 0x562e2bb7
Variable types: 322500 continuous, 3225 integer (3225 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+02]
  Objective range  [1e-02, 1e-02]
  Bou

Variable types: 322500 continuous, 3225 integer (3225 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+02]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 318976 rows and 318976 columns
Presolve time: 0.50s
Presolved: 3525 rows, 6749 columns, 18424 nonzeros
Variable types: 0 continuous, 6749 integer (6749 binary)
Found heuristic solution: objective 11.5700000

Root relaxation: objective 1.629211e+02, 68 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  162.92107    0    6   11.57000  162.92107  1308%     -    0s
H    0     0                     162.3100000  162.92107  0.38%     -    0s
H    0     0                     162.8000000  162.92107  0.07%     -    0s
     0     0  162.88730    0   17  162.800

     0     0  162.93593    0   10  162.87000  162.93593  0.04%     -    0s
     0     0  162.93582    0   10  162.87000  162.93582  0.04%     -    0s
     0     0  162.93314    0    9  162.87000  162.93314  0.04%     -    0s
     0     0  162.93300    0   10  162.87000  162.93300  0.04%     -    0s
     0     0  162.93157    0    9  162.87000  162.93157  0.04%     -    0s
     0     0  162.93123    0   10  162.87000  162.93123  0.04%     -    0s
     0     0  162.93107    0   11  162.87000  162.93107  0.04%     -    0s
     0     0  162.92900    0    9  162.87000  162.92900  0.04%     -    0s
     0     0  162.92595    0    9  162.87000  162.92595  0.03%     -    0s
     0     0  162.92536    0    9  162.87000  162.92536  0.03%     -    0s
     0     2  162.92536    0    9  162.87000  162.92536  0.03%     -    0s

Cutting planes:
  Gomory: 2
  Cover: 1
  MIR: 6
  StrongCG: 1

Explored 78 nodes (288 simplex iterations) in 0.87 seconds (0.55 work units)
Thread count was 12 (of 12 availab

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [01:44<00:00,  5.25s/it]


CPU time: 1061.9774547999987
95% Confidence Interval of upper bound estimate: [162.59, 163.09]
95% Confidence Interval of lower bound estimate: [143.99, 144.83]
Estimated optimality gap is:  11.316 %


Experiment 2.3: Varying num edges from 5 to 100. Num edges - fixed as 2000; B = 200.

In [304]:
# Run the algo with num_edges in [50, 100, 200, 500, 1000, 2000, 5000, 10000]
num_sample = 20
num_valid = 500
num_test = 500
B = 200
num_train = 100

outputs = []
with open('output3.txt', 'w') as f:
    for num_edges in [50, 100, 200, 500, 1000, 2000, 5000, 10000]:
        print(f'running experiment num_edges = {num_edges}....')
        # Choose a random subgraph with the specified number of edges
        random.seed(10)
        subgraph_edges = random.sample(G2.edges(), num_edges)
        sub_G2 = G2.edge_subgraph(subgraph_edges)
        print('Num nodes:', len(sub_G2))
        print('Num edges:', len(sub_G2.edges))
        # Add random activation probabilities for each edge
        np.random.seed(0)
        for e in sub_G2.edges():
            sub_G2[e[0]][e[1]]['weight'] = np.random.uniform(0, 1)

        np.random.seed(0)
        # Cost of each node is uniformly distribution from (0, 100)
        C = {v: np.random.uniform(0, 100) for v in sub_G2.nodes}
        
        output = main_algo(sub_G2, B, C, num_sample, num_train, num_valid, num_test)
        f.write(f'Output for num_edges = {num_edges}:\n')
        for item in output:
            f.write(str(item))
            f.write('\n')
        f.write('\n')
        outputs.append(output)

running experiment num_edges = 50....


C:\Users\USER\AppData\Local\Temp\ipykernel_18940\2335898128.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subgraph_edges = random.sample(G2.edges(), num_edges)


Num nodes: 100
Num edges: 50
currently in SAA phase 1 1th sample
finding predecessors for scenario 1
finding predecessors for scenario 2
finding predecessors for scenario 3
finding predecessors for scenario 4
finding predecessors for scenario 5
finding predecessors for scenario 6
finding predecessors for scenario 7
finding predecessors for scenario 8
finding predecessors for scenario 9
finding predecessors for scenario 10
finding predecessors for scenario 11
finding predecessors for scenario 12
finding predecessors for scenario 13
finding predecessors for scenario 14
finding predecessors for scenario 15
finding predecessors for scenario 16
finding predecessors for scenario 17
finding predecessors for scenario 18
finding predecessors for scenario 19
finding predecessors for scenario 20
finding predecessors for scenario 21
finding predecessors for scenario 22
finding predecessors for scenario 23
finding predecessors for scenario 24
finding predecessors for scenario 25
finding predecessor


CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10001 rows, 10100 columns and 22419 nonzeros
Model fingerprint: 0xba921c91
Variable types: 10000 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+02]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 9950 rows and 9950 columns
Presolve time: 0.01s
Presolved: 51 rows, 150 columns, 250 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)
Found heuristic solution: objective 11.3900000

Root relaxation: objective 2.852825e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   28.52


CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10001 rows, 10100 columns and 22404 nonzeros
Model fingerprint: 0x693f1943
Variable types: 10000 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+02]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 9950 rows and 9950 columns
Presolve time: 0.01s
Presolved: 51 rows, 150 columns, 250 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)
Found heuristic solution: objective 11.2700000

Root relaxation: objective 2.822737e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   28.22

...

Model fingerprint: 0x0daf5c97
Variable types: 1186400 continuous, 11864 integer (11864 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+02]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 1109000 rows and 1109003 columns (presolve time = 5s) ...
Presolve removed 1108980 rows and 1108983 columns
Presolve time: 7.62s
Presolved: 77421 rows, 89281 columns, 1155863 nonzeros
Variable types: 0 continuous, 89281 integer (89261 binary)
Found heuristic solution: objective 23.5700000
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.81s

Barrier performed 0 iterations in 11.17 seconds (12.50 work units)
Barrier solve interrupted - model solved by another algorithm

Concurrent spin time: 0.01s

Solved with dual simplex

Root simplex log...

Iteration    Objective       Pri

finding predecessors for scenario 11
finding predecessors for scenario 12
finding predecessors for scenario 13
finding predecessors for scenario 14
finding predecessors for scenario 15
finding predecessors for scenario 16
finding predecessors for scenario 17
finding predecessors for scenario 18
finding predecessors for scenario 19
finding predecessors for scenario 20
finding predecessors for scenario 21
finding predecessors for scenario 22
finding predecessors for scenario 23
finding predecessors for scenario 24
finding predecessors for scenario 25
finding predecessors for scenario 26
finding predecessors for scenario 27
finding predecessors for scenario 28
finding predecessors for scenario 29
finding predecessors for scenario 30
finding predecessors for scenario 31
finding predecessors for scenario 32
finding predecessors for scenario 33
finding predecessors for scenario 34
finding predecessors for scenario 35
finding predecessors for scenario 36
finding predecessors for scenario 37
f

finding predecessors for scenario 65
finding predecessors for scenario 66
finding predecessors for scenario 67
finding predecessors for scenario 68
finding predecessors for scenario 69
finding predecessors for scenario 70
finding predecessors for scenario 71
finding predecessors for scenario 72
finding predecessors for scenario 73
finding predecessors for scenario 74
finding predecessors for scenario 75
finding predecessors for scenario 76
finding predecessors for scenario 77
finding predecessors for scenario 78
finding predecessors for scenario 79
finding predecessors for scenario 80
finding predecessors for scenario 81
finding predecessors for scenario 82
finding predecessors for scenario 83
finding predecessors for scenario 84
finding predecessors for scenario 85
finding predecessors for scenario 86
finding predecessors for scenario 87
finding predecessors for scenario 88
finding predecessors for scenario 89
finding predecessors for scenario 90
finding predecessors for scenario 91
f

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [06:32<00:00, 19.61s/it]


CPU time: 3389.626702200003
95% Confidence Interval of upper bound estimate: [616.88, 620.84]
95% Confidence Interval of lower bound estimate: [463.56, 469.46]
Estimated optimality gap is:  24.618 %
